<a target="_blank" href="https://colab.research.google.com/github/leonardocrociani/MCTS-Pokemon-Battle-Policy">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# **MCTS Pokémon VGC Competition**  

_Course: Artificial Intelligence Fundamentals, A.Y. 2024/25_  
_Author: Leonardo Crociani_  
_Github Repository: [MCTS-Pokemon-Battle-Policy](https://github.com/leonardocrociani/MCTS-Pokemon-Battle-Policy)_

<small>**Important Notice:** The framework's creator announced on the official Discord channel that a new engine version is expected in the third week of January. If this update includes API changes, some code discrepancies may arise. This code was developed using version **3.0.5.7** of the master branch.</small>

[Are you searching the runnable-code version?](https://colab.research.google.com/github/leonardocrociani/MCTS-Pokemon-Battle-Policy/blob/main/AIF_runnable_code.ipynb)

## **Introduction**  

For the Artificial Intelligence Fundamentals (AIF) exam, I chose to work on the **Pokémon VGC AI competition project**, specifically pursuing the **Battle Policy track**.  

The implemented battle policy interacts with a predefined environment—a highly convenient framework [1]—that manages the game state and its updates, allowing the user to focus solely on policy development.  

**The goal of the policy is to defeat any other battle policy, regardless of the team composition used.**  

The game of **Pokémon** is particularly interesting. When considering teams of three Pokémon, the branching factor for each player's possible states is at most **6** (four attack moves and 2 switches). Both players select their moves simultaneously, and the new game state is then evaluated.

There are multiple factors to take into account:

1. An agent may encounter issues such as infinite looping, continuously selecting the `switch` action.  

2. In the short term (i.e., looking one turn ahead), we can make a reasonable estimation of the new state, but in the long term, this estimation loses significance due to several factors:  
  a. The game incorporates **probability**, introducing randomness into outcomes.  
  b. During the initial phases, the game is **not fully observable**, meaning we might complete a battle without ever having full knowledge of the opponent’s team.  

For these reasons, I decided to implement a **Monte Carlo Tree Search (MCTS)-based Battle Policy**. My goal was to leverage the robustness of **MCTS**, complemented by heuristics to guide the playouts and the node selection effectively.  

## **Related Work**  

The application of artificial intelligence in Pokémon Video Game Championships has been explored through various methodologies, notably Monte Carlo Tree Search and Reinforcement Learning.  

MCTS has been successfully utilized in developing AI agents for Pokémon battles. **Norström (2019)** identified MCTS as an effective algorithm for full-scale Pokémon battles, emphasizing its balance between exploration and exploitation in decision-making processes [2].  

Additionally, the implementation of Information Set Monte Carlo Tree Search has been proposed by **Ihara H., et al. (2018)** [3] to address the imperfect information scenarios inherent in Pokémon battles, aiming to mitigate strategy fusion issues caused by determinization.  

RL approaches have also been applied to develop competitive Pokémon battle strategies. **Kalose and Kaya (2018)** [4] explored RL techniques to determine optimal battle strategies, demonstrating RL's potential in handling the complexities of Pokémon battles.  

Moreover, the integration of deep reinforcement learning and neuroevolution has been investigated by **Rodriguez, G., et al. (2024)** [5] to enhance AI performance in the VGC context, highlighting the adaptability of these methods to the game's dynamic environment.

## **Methodologies**  

The Monte Carlo Tree Search algorithm runs an arbitrary number of simulations from the current game state and selects the **most promising** action.  

The algorithm consists of four phases:  
- *Selection*: Identifies the most suitable node to expand within the current tree of states.  
- *Expansion*: Expands the selected node by adding new child nodes.  
- *Simulation*: Simulates gameplay from the newly expanded node down to a terminal state.  
- *Backpropagation*: Propagates the simulation results back up the tree to update node values.  


```python
def mcts_search(self, root):
    for _ in tqdm(range(self.max_iterations), desc='One turn simulations'):
        current = root
        # Selection
        while current.children and current.is_fully_expanded():
            current = max(current.children, key=self.ucb1)
        # Expansion
        if not current.is_fully_expanded():
            current = self.expand(current)
        # Simulation
        reward = self.playout(current)
        # Backpropagation
        self.back_propagate(current, reward)
        
    if len(root.children) == 0:        
        if self.debug:
            print('No children, going random')
        return random.randint(0, DEFAULT_N_ACTIONS - 1)
        
    best_action = max(root.children, key=lambda c: c.reward / (c.visits + 1e-6), default=None)
        
    if self.debug:
        if best_action:
            print(f"Best action: {best_action.connection_action}, Reward: {best_action.reward}, Visits: {best_action.visits}")
        else:
            print('No best action found, going random')
            
    return best_action.connection_action if best_action else random.randint(0, DEFAULT_N_ACTIONS - 1)
```

### *Selection*  

The selection phase is performed using the **UCB1** formula:  

$UCB1 (n) = \frac{U(n)}{N(n)} + C \cdot \sqrt{\frac{\log N (\text{Parent}(n))}{N(n)}}$  

Where:  
- $U(n)$ is the total utility of all playouts that passed through node $n$.  
- $N(n)$ is the total number of playouts that passed through node $n$.  
- $C$ is a balancing factor between the left term (_"exploitation"_ term) and the right term (_"exploration"_ term).  
- $\text{Parent}(n)$ is the parent node of $n$ in the tree.  

Typically, $C$ is set to $\sqrt{2}$, but as stated in the AIMA book, game developers often fine-tune this parameter.  

This is precisely what I did! Through experimentation, I found that a suitable value for $C$ is **10**. Additionally, since the early stages of the game provide limited information about the opponent, $C$ changes dynamically over time.  

In the initial turns, I prioritize _exploration_, aiming to identify a balanced move—one that is not necessarily optimized but generally effective. However, $C$ rapidly decreases to $10$, following the formula defined in the `get_action` method of the battle policy:  

```python
self.C = max(self.min_C, self.max_C / (self.turns ** 1.2))


### *Expansion*  

From the root node and throughout the entire tree, we must decide how to expand nodes. The standard MCTS algorithm expands by considering all possible actions. This was the approach I initially implemented in the first version of the code. However, after realizing that this approach was not optimal, I transitioned to a **pruned** version.  

Over successive development iterations, **pruning** evolved into a strategic component of the algorithm.  

The goal is to **avoid suboptimal actions**. In the code snippet below, the `is_action_suboptimal` function is used to prune ineffective actions based on factors such as the **opponent’s Pokémon types, attack stage, and weather conditions**.

```python
def is_action_suboptimal(game_state: GameState, action: int, effectiveness_threshold: float = 0.5) -> bool:
    active_moves = game_state.teams[0].active.moves

    # check if the action is a switch
    if action >= len(active_moves):
        switch_index = action - len(active_moves)
        my_team = game_state.teams[0]
        
        if switch_index >= len(my_team.party) or my_team.party[switch_index].hp <= 0:
            return True  # avoid switching to a fainted pokemon

        # let's eval the matchup of the current pokemon and of the switch pokemon
        current_matchup = match_up_eval(
            my_team.active.type,
            game_state.teams[1].active.type,
            [m.type for m in my_team.active.moves],
            [m.type for m in game_state.teams[1].active.moves if m.name]
        )

        switch_matchup = match_up_eval(
            my_team.party[switch_index].type,
            game_state.teams[1].active.type,
            [m.type for m in my_team.party[switch_index].moves],
            [m.type for m in game_state.teams[1].active.moves if m.name]
        )

        # if the switch doesn't improve the matchup by a threshold, consider it suboptimal
        is_switch_suboptimal = (switch_matchup - SWITCH_THRESHOLD) < current_matchup
        return is_switch_suboptimal

    # for the attack moves let's compute the expected damage with estimate_damage() [from @thunder battle policy]
    move = active_moves[action]
    attacker = game_state.teams[0].active
    defender = game_state.teams[1].active
    
    my_team = game_state.teams[0]
    my_attack_stage = my_team.stage[PkmStat.ATTACK]

    opp_team = game_state.teams[1]
    opp_defense_stage = opp_team.stage[PkmStat.DEFENSE]

    estimated_dmg = estimate_damage(
        move=move,
        pkm_type=attacker.type,
        opp_pkm_type=defender.type,
        attack_stage=my_attack_stage,
        defense_stage=opp_defense_stage,
        weather=game_state.weather
    )

    # if the damage is too low, i'll consider the move suboptimal
    defender_max_hp = defender.max_hp
    damage_ratio = estimated_dmg / defender_max_hp  # percentage of removed hp

    is_suboptimal = damage_ratio < DAMAGE_THRESHOLD or estimated_dmg <= effectiveness_threshold
    return is_suboptimal
```

```python
def estimate_damage(move: PkmMove, pkm_type: PkmType, opp_pkm_type: PkmType,
                    attack_stage: int, defense_stage: int, weather: WeatherCondition) -> float:
    move_type: PkmType = move.type
    move_power: float = move.power
    type_rate = TYPE_CHART_MULTIPLIER[move_type][opp_pkm_type]
    if type_rate == 0:
        return 0
    if move.fixed_damage > 0:
        return move.fixed_damage
    stab = 1.5 if move_type == pkm_type else 1.
    if (move_type == PkmType.WATER and weather == WeatherCondition.RAIN) or (
            move_type == PkmType.FIRE and weather == WeatherCondition.SUNNY):
        weather = 1.5
    elif (move_type == PkmType.WATER and weather == WeatherCondition.SUNNY) or (
            move_type == PkmType.FIRE and weather == WeatherCondition.RAIN):
        weather = .5
    else:
        weather = 1.
    stage_level = attack_stage - defense_stage
    stage = (stage_level + 2.) / 2 if stage_level >= 0. else 2. / \
        (np.abs(stage_level) + 2.)
    damage = type_rate * \
        stab * weather * stage * move_power
    return damage
```

### *Simulation (Playout)*  

Now that we have selected a non-suboptimal node, we must simulate the entire game until reaching a terminal state.  

#### *Playout Policy*  

To guide the simulation, we need to define a **playout policy**, which determines how actions are chosen sequentially.  

The traditional playout strategy follows a **purely random** approach.  

Each transition between game states can only be triggered if both our battle policy and the opponent's policy provide an action.  

The playout policy in my implementation follows a strategy where it **creates a pool of non-suboptimal moves** and then **selects one randomly**, leveraging the intrinsic robustness of MCTS.  

*What about the opponent?*

In an earlier versions of the code, I implemented a strategy where the **opponent followed the same selection process** as our policy. However, this approach had limitations due to the lack of information available about the opponent.  

Moreover, empirical testing showed poor results with this method.  

In the latest version, the opponent plays **purely random** moves.  

#### *Number of Simulations*  

The number of simulations was tuned to achieve a competitive MCTS based battle policy. After extensive testing, I found that an optimal number of simulations is:  

$\textbf{O(10^5)}$  

Each search can take anywhere from one to 10 minutes, depending on the pruning of suboptimal actions. Since the framework (as also confirmed in [1]) does not specify any time constraints, I considered a 10-minute runtime acceptable.  

### *Backpropagation and Utility*  

When the simulation reaches a terminal state, a **utility function** evaluates whether the outcome was favorable.  

A simple scoring system, such as assigning 1 for a win and -1 for a loss, would be an oversimplification.  

To improve this evaluation, I incorporated additional factors, including:  
- **Matchup score**  
- **Remaining health points**  
- **Pokémon status conditions** (e.g., poisoned, paralyzed, etc.)  
- **And more...** (Refer to the `evaluate_terminal_state()` function for a complete list.)  

These factors are combined into a weighted sum.  

Once the utility is calculated, the **results are backpropagated** through the tree up to the root node.  


```python
def evaluate_terminal_state(state: GameState, cycles: int, num_switches: int) -> float:
    my_team = state.teams[0]
    opp_team = state.teams[1]
    my_active: Pkm = my_team.active
    opp_active: Pkm = opp_team.active

    # base matchup evaluation
    match_up = evaluate_actives_matchup(
        my_active,
        opp_active
    )

    # hp evaluation
    my_hp_ratio = my_active.hp / my_active.max_hp
    opp_hp_ratio = opp_active.hp / opp_active.max_hp
    
    # hp is more valuable when it's lower
    hp_weight = 4.0
    my_hp_score = hp_weight * (1 + (1 - my_hp_ratio)) * my_hp_ratio
    opp_hp_score = hp_weight * (1 + (1 - opp_hp_ratio)) * opp_hp_ratio

    # team health evaluation
    my_team_health = 0
    for pokemon in [my_team.party[0], my_team.party[1]]:
        if pokemon.hp > 0:
            health_ratio = pokemon.hp / pokemon.max_hp
            # backup pokemon health is important
            my_team_health += health_ratio * 2  

    my_status_score = detailed_status_eval(my_active)
    opp_status_score = detailed_status_eval(opp_active)

    my_stage_score = evaluate_stages(my_team)
    opp_stage_score = evaluate_stages(opp_team)


    # weather
    weather_score = 0
    if state.weather != WeatherCondition.CLEAR:
        if (state.weather == WeatherCondition.SUNNY and my_active.type == PkmType.FIRE) or \
           (state.weather == WeatherCondition.RAIN and my_active.type == PkmType.WATER):
            weather_score += 0.5
        elif (state.weather == WeatherCondition.SUNNY and opp_active.type == PkmType.FIRE) or \
             (state.weather == WeatherCondition.RAIN and opp_active.type == PkmType.WATER):
            weather_score -= 0.5

    # switching penalty
    switch_penalty = -0.15 * num_switches * max(0, (30 - cycles) / 30)

    utility = (
        match_up * 1.2 +                    # matchup
        my_hp_score - opp_hp_score +        # active pkm hp
        my_team_health * 0.8 +              # team health
        my_status_score - opp_status_score + # status conditions
        my_stage_score * 0.3 -              # stat stages
        opp_stage_score * 0.3 +             # opp stat stages
        weather_score +                      # weather effects
        switch_penalty                       # switch penalty
    )

    return utility
```

## **Assessment**  

The evaluation of the battle policy was conducted against four different pre-built policies, arranged in increasing order of complexity:  
1. `TypeSelector` policy  
2. `PrunedBFS` policy  
3. `Minimax` policy  
4. `TunedTreeSearch` policy  

Although the original competition rules state that each match should consist of 10 battles with a team switch halfway through, I opted for three tests.

Each test consisted of two games, with teams being swapped at the end of each game to eliminate potential team imbalance.  

Each game comprised two or three battles, depending on the match outcomes.

Here's the Results (rounded to the nearest integer):  

| Opponent Policy      | MCTS Win Rate | Avg. No. of Turns | Total Battles |
|--------------------|---------|-------------------|--------------|
| `TypeSelector`   | 79%  | 7             | 14       |
| `PrunedBFS`      | 64%  | 6             | 14       |
| `Minimax`        | 100%  | 8             | 12        |
| `TunedTreeSearch`| 64%  | 8             |14       |


- Against `Minimax` the MCTS achieved an **100% win rate**, suggesting that our policy effectively countered its decision-making process despite longer battles (8 turns on average).  
- Against the `TypeSelector` policy, the **79% win rate** indicates that the pruning strategy was effective in exploiting type-based advantages.  
- `PrunedBFS` and the  `TunedTreeSearch` policies posed more of a challenge (**64% win rate**).


## **Conclusion**  

I thoroughly enjoyed working on this project. The Monte Carlo Tree Search method proved to be an powerful tool for decision-making in Pokémon battles. Furthermore, integrating heuristics significantly enhanced its effectiveness, played an important role in guiding MCTS, allowing it to prune suboptimal moves and focus on more promising strategies.

This refinement not only improved efficiency but also reduced unnecessary or ineffective actions.  

At the same time, the inherent randomness in MCTS contributed to the robustness of the policy, making it adaptable to the stochastic nature of Pokémon battles.

The combination of strategic pruning through heuristics and randomized exploration created a well-balanced approach capable of handling uncertainty and imperfect information.  

This project enhanced my understanding of game AI, decision-making under uncertainty, and heuristic-driven search methods. Of course, there is still considerable room for improvement. I am especially interested in integrating reinforcement learning, as its potential deeply fascinates me.

## **Citations**  

[1] Reis, S., et al. (2021). *VGC AI Competition - A New Model of Meta-Game Balance AI Competition* [IEEE Conference Of Games](https://ieee-cog.org/2021/assets/papers/paper_6.pdf) | *Pokémon VGC Engine Repository* [GitLab](https://gitlab.com/DracoStriker/pokemon-vgc-engine).

[2] Norström, J. (2019). *Monte Carlo Tree Search in Pokémon Battles: An Empirical Study*. [Tilburg University Repository](https://arno.uvt.nl/show.cgi?fid=170059)  

[3] Ihara H., et al. (2018). *Implementation and Evaluation of Information Set Monte Carlo Tree Search for Pokémon*. [IEEE Xplore](https://ieeexplore.ieee.org/document/8616371/)  

[4] Kalose, K., & Kaya, M. (2018). *Optimal Battle Strategy in Pokémon using Reinforcement Learning*. [Semantic Scholar](https://www.semanticscholar.org/paper/Optimal-Battle-Strategy-in-Pok%C3%A9mon-using-Learning-Kalose-Kaya/897f281adb99d158c2f53fb68d0e20ea510a7cab)  

[5] Rodriguez, G., et al. (2024). *Enhancing Pokémon VGC Player Performance: Intelligent Agents Through Deep Reinforcement Learning and Neuroevolution*. [ACM Digital Library](https://dl.acm.org/doi/10.1007/978-3-031-60692-2_19)  
